# Scenario 1: Spearphishing Email with a Malware Link
### Description:
A spearphishing email was sent to a company employee, pretending to be from their IT department. The email urges the recipient to click on a link to download a "security update," which actually downloads a malware payload.

## Attributes to Include:
- Email source
- Email subject
- Sender IP address
- URL leading to the malware download
- Malware file hashes and filename
- Sandbox analysis links

### Indicators:
| Description             | Category          | Type          | Value                                                            |
|-------------------------|-------------------|---------------|------------------------------------------------------------------|
| Email sender            | Payload delivery  | email-src     | it-support@fakesecurity.com                                      |
| Email subject           | Payload delivery  | email-subject | Urgent: Security Update Required                                 |
| Sender IP address       | Payload delivery  | ip-src        | 203.0.113.50                                                     |
| Malicious URL           | Payload delivery  | url           | http://fake-update.com/security-update.exe                       |
| File Name               | Network activity  | filename      | security-update.exe                                              |
| Malware Hash (MD5)      | Network activity  | md5           | a1b2c3d4e5f67890123456789abcdef0                                 |
| Malware Hash (SHA256)   | Network activity  | sha256        | f65ac3e2b9e6f9dbb0978d2389e324f85d65f3e11c8b6e98efcfd7685f123abc |
| Online sandbox analysis | External analysis | link          | https://my.sandbox.com/analysis/suspicious-url-0001              |

## Galaxy Clusters to Include:
### Attack Pattern
 - Phishing (Spearphishing Link):
   - Technique ID: T1566.002
   - Description: Phishing email contains a link to download malware.

# Initialize environment
This section initializes the playbook environment and loads the required Python libraries. The credentials for MISP (authkey) is loaded from the file `keys.py` in the directory **vault**. A PyMISP object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem, PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least:
```
misp_url = "<MISP URL>"             # The URL to our MISP server
misp_key = "<MISP Authkey>"         # The MISP authkey
misp_verifycert = <True or False>   # Indicate if PyMISP should attempt to verify the certificate or ignore errors

In [ ]:
from vault.keys import *

from pymisp import PyMISP, MISPEvent, MISPObject


if not misp_verifycert:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))


# Initialize MISP Event
This cell will initialize the MISP Event and attach the Event metadata.

In [ ]:
event = MISPEvent()
event.info = "<Title of MISP Event>"

## Threat Level
# 1 - High means sophisticated APT malware or 0day attack
# 2 - Medium means APT malware
# 3 - Low means mass malware
# 4 - Undefine

event.threat_level_id = <ID>


## Analysis
# 0 - Initial means Event has just been created and is in an initial state
# 1 - Ongoing means the analysis is still ongoing
# 2 - Completed means the Event creator considers the analysis complete

event.analysis = <ID>


## Distribution 
# 0 - Your organization means only members of your organization on this server will be able to see this Event
# 1 - This community only means organizations that are a part of this community server will be able to see this Event
# 2 - Connected communities means organizations that are a part of this community server or a part of a connected (synced) community MISP server will be able to see the event

event.distribution = <ID>

# Initialize Indicator Variables
This cell will initialize the indicator variables we will use to add Attributes to the Event.

In [ ]:
# Copy the indicators from the table above into the appropriate indicator variables

email_sender = ""
email_subject = ""
sender_ip = ""
malicious_url = ""
sandbox_link = ""
malware_filename = ""
malware_md5 = ""
malware_sha256 = ""

# Add Attributes to the Event
This cell will add attributes to the Event.

In [ ]:
event.add_attribute(type="email-src", value=email_sender, category="Payload delivery")
event.add_attribute(type="email-subject", value=email_subject, category="Payload delivery")
event.add_attribute(type="ip-src", value=sender_ip, category="Payload delivery")
event.add_attribute(type="url", value=malicious_url, category="Payload delivery", comment="embedded URL")
event.add_attribute(type="link", value=sandbox_link, category="External analysis")

# For added context, you can add a comment attribute that provides a brief description of the intel share
description = """
A spearphishing email was sent to a company employee, pretending to be from their IT department. 
The email urges the recipient to click on a link to download a "security update," 
which actually downloads a malware payload.
"""
event.add_attribute(type="comment", value=description)

# Initialize and add Attributes to MISP Object
This cell will initialize and add attributes to a MISP Object.

In [ ]:
file_object = MISPObject(name="file")
file_object.comment = "malicious file downloaded from phishing link"

file_object.add_attribute(object_relation="filename", simple_value=malware_filename)
file_object.add_attribute(object_relation="md5", simple_value=malware_md5)
file_object.add_attribute(object_relation="sha256", simple_value=malware_sha256)

# Add Object to Event
event.add_object(file_object)

# Contextualize the Event with Galaxy Clusters
This cell will add context to the Event by utilizing galaxy cluster tags.

In [ ]:
# Attack Pattern
event.add_tag('misp-galaxy:mitre-attack-pattern="Spearphishing Link - T1566.002"')

# Publish and Share the Event
This cell will publish and share the event to the MISP server.

In [ ]:
event.publish()

misp.add_event(event)